In [1]:

import pickle
import numpy as np
import tensorflow as tf
import time
import math

from matplotlib import pyplot as plt
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix
from datetime import timedelta
from keras.datasets import cifar10

import keras
from keras import backend as K
#from Freeze_Graph_Script import freeze_graph

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import Adagrad
from keras.optimizers import SGD
from keras.constraints import maxnorm

%matplotlib inline

/Users/Divija/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Loading data into numpy arrays
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
train_images = x_train.astype('float32') / 255.0
test_images = x_test.astype('float32') / 255.0

In [4]:
train_images.shape

(50000, 32, 32, 3)

In [5]:
test_images.shape

(10000, 32, 32, 3)

In [6]:
y_train.shape

(50000, 1)

In [7]:
#Flattened y labels train
train_labels = np_utils.to_categorical(y_train)

In [8]:
#Flattened y labels train
test_labels = np_utils.to_categorical(y_test)

In [9]:
train_labels.shape

(50000, 10)

In [10]:
test_labels.shape

(10000, 10)

In [11]:
# Function to reshape each image (32 x 32 x 3) into 1 long array (1 x 3072)
def reshape_image(input_array):
    output_array = []
    for image in input_array:
        output_array.append(image.reshape(-1))
    return np.asarray(output_array)

In [12]:
train_images = reshape_image(train_images)

In [13]:
test_images = reshape_image(test_images)

In [14]:
train_images.shape

(50000, 3072)

In [15]:
# Sets the learning phase so that we don't have to enter a value for dropout in android
K.set_learning_phase(1)

In [16]:
# Set up the sequential type model (just add layers in the order you want them to be executed
model = Sequential()


In [17]:
# Reshape the input into a 32 x 32 matrix with 3 color channels for more accurate analysis
model.add(Reshape((32, 32, 3), input_shape=(3072,)))


In [18]:
# Associate 5 x 5 patches of each image with 32 features (because we are using float32) with relu activation function
model.add(Conv2D(32, (5, 5), input_shape=(32, 32, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))


In [19]:
# Get the max value of each 2 x 2 block and discard the rest, thus changing image size to 16 x 16
model.add(MaxPooling2D(pool_size=(2, 2)))


In [20]:
# Associate 5 x 5 patches of each image with 32 features (because we are using float32) with relu activation function
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))


In [21]:
# Get the max value of each 2 x 2 block and discard the rest, thus changing image size to 16 x 16
model.add(MaxPooling2D(pool_size=(2, 2)))


In [22]:
# Flatten the matrix into an array
model.add(Flatten())


In [23]:
# Dense layer creates 512 connected neurons (32 x 16) with relu activation function
model.add(Dense(1024, activation='softmax', kernel_constraint=maxnorm(3)))


In [24]:
# Dropout layer prevents overfitting by ignoring outputs from some neurons
model.add(Dropout(0.5))


In [25]:
# Last dense layer sorts the features into 10 possible outputs with softmax activation
model.add(Dense(10, activation='softmax'))



In [26]:
# Specify loss function, optimizer, and which metrics we want to keep track of
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01), metrics=['accuracy'])


In [27]:
from keras.callbacks import TensorBoard
tensorboard_1 = TensorBoard(log_dir="Desktop\Tensorboard\model_1_GPU")

Instructions for updating:
Use the retry module or similar alternatives.


In [28]:
model.fit(train_images, train_labels, epochs=10, batch_size=32, callbacks=[tensorboard_1])

Epoch 1/10
50000/50000 [==============================] - 213s 4ms/step - loss: 2.3027 - acc: 0.0992
Epoch 2/10
50000/50000 [==============================] - 189s 4ms/step - loss: 2.3027 - acc: 0.0972
Epoch 3/10
50000/50000 [==============================] - 197s 4ms/step - loss: 2.3027 - acc: 0.0966
Epoch 4/10
50000/50000 [==============================] - 194s 4ms/step - loss: 2.3027 - acc: 0.0980
Epoch 5/10
50000/50000 [==============================] - 179s 4ms/step - loss: 2.3027 - acc: 0.0974
Epoch 6/10
50000/50000 [==============================] - 184s 4ms/step - loss: 2.3027 - acc: 0.0996
Epoch 7/10
50000/50000 [==============================] - 176s 4ms/step - loss: 2.3027 - acc: 0.0981
Epoch 8/10
50000/50000 [==============================] - 179s 4ms/step - loss: 2.3027 - acc: 0.0982
Epoch 9/10
50000/50000 [==============================] - 181s 4ms/step - loss: 2.3027 - acc: 0.0991
Epoch 10/10
50000/50000 [==============================] - 195s 4ms/step - loss: 2.3027 - a

In [29]:
#Predict against test
pred = model.predict(test_images)

In [30]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [31]:
#Convert for accuracy calculation
pred_vals = pred.argmax(axis=1)
true_vals = test_labels.argmax(axis=1)

In [32]:
accuracy_score(true_vals, pred_vals)

0.1018

In [33]:
confusion_matrix(true_vals, pred_vals)

array([[889,   0,   0,   2,   1,   0,   4,   2, 102,   0],
       [857,   0,   0,   0,   3,   0,  11,   2, 127,   0],
       [880,   0,   0,   3,   3,   0,   6,   1, 107,   0],
       [881,   0,   0,   2,   1,   0,   6,   3, 107,   0],
       [888,   0,   0,   4,   1,   0,   4,   1, 102,   0],
       [880,   0,   0,   3,   0,   0,   2,   1, 114,   0],
       [871,   0,   0,   4,   0,   0,   7,   3, 115,   0],
       [872,   0,   0,   3,   1,   0,   5,   2, 117,   0],
       [871,   0,   0,   2,   1,   1,   4,   4, 117,   0],
       [891,   0,   0,   2,   2,   0,   4,   0, 101,   0]])